# Assignment 2 - Housing Regression

### Part 1 (and some Part 2) - Transforms, and looking at data
My computer took forever to even make a scatter matrix using only five columns, so I ended going into this a little bit 'blind' so to speak considering my lack of visual aid. However, by looking at the names of columns, I think that a few variables will have an effect on the target variable, just based on my (small) prior knowledge of real estate.

I think lot size will have a positive effect on price. I think pool area will have a positive effect on price. To be more specific, I think entries with a pool area higher than 0 will have a better price than entries with a pool area of 0. I think that the newer the house is (year built) the better the price will be.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from IPython.display import display
pd.options.display.max_columns = None
from sklearn.metrics import r2_score
from sklearn import impute
from sklearn import preprocessing
import numpy as np
from sklearn.feature_selection import RFECV
from sklearn.metrics import make_scorer

In [2]:
data = pd.read_csv('AmesHousingSetA.csv')
data.shape

(2344, 81)

In [3]:
#looking at the columns, trying to find some sense of what is going on
print(data.columns)
print(data.shape)
print(data.dtypes)
#reorder to put the target as the first column, makes it easier down the line
data = data[['SalePrice', 'PID', 'MS.SubClass', 'MS.Zoning', 'Lot.Frontage', 'Lot.Area', 'Street',
       'Alley', 'Lot.Shape', 'Land.Contour', 'Utilities', 'Lot.Config',
       'Land.Slope', 'Neighborhood', 'Condition.1', 'Condition.2', 'Bldg.Type',
       'House.Style', 'Overall.Qual', 'Overall.Cond', 'Year.Built',
       'Year.Remod.Add', 'Roof.Style', 'Roof.Matl', 'Exterior.1st',
       'Exterior.2nd', 'Mas.Vnr.Type', 'Mas.Vnr.Area', 'Exter.Qual',
       'Exter.Cond', 'Foundation', 'Bsmt.Qual', 'Bsmt.Cond', 'Bsmt.Exposure',
       'BsmtFin.Type.1', 'BsmtFin.SF.1', 'BsmtFin.Type.2', 'BsmtFin.SF.2',
       'Bsmt.Unf.SF', 'Total.Bsmt.SF', 'Heating', 'Heating.QC', 'Central.Air',
       'Electrical', 'X1st.Flr.SF', 'X2nd.Flr.SF', 'Low.Qual.Fin.SF',
       'Gr.Liv.Area', 'Bsmt.Full.Bath', 'Bsmt.Half.Bath', 'Full.Bath',
       'Half.Bath', 'Bedroom.AbvGr', 'Kitchen.AbvGr', 'Kitchen.Qual',
       'TotRms.AbvGrd', 'Functional', 'Fireplaces', 'Fireplace.Qu',
       'Garage.Type', 'Garage.Yr.Blt', 'Garage.Finish', 'Garage.Cars',
       'Garage.Area', 'Garage.Qual', 'Garage.Cond', 'Paved.Drive',
       'Wood.Deck.SF', 'Open.Porch.SF', 'Enclosed.Porch', 'X3Ssn.Porch',
       'Screen.Porch', 'Pool.Area', 'Pool.QC', 'Fence', 'Misc.Feature',
       'Misc.Val', 'Mo.Sold', 'Yr.Sold', 'Sale.Type', 'Sale.Condition']]


Index(['PID', 'MS.SubClass', 'MS.Zoning', 'Lot.Frontage', 'Lot.Area', 'Street',
       'Alley', 'Lot.Shape', 'Land.Contour', 'Utilities', 'Lot.Config',
       'Land.Slope', 'Neighborhood', 'Condition.1', 'Condition.2', 'Bldg.Type',
       'House.Style', 'Overall.Qual', 'Overall.Cond', 'Year.Built',
       'Year.Remod.Add', 'Roof.Style', 'Roof.Matl', 'Exterior.1st',
       'Exterior.2nd', 'Mas.Vnr.Type', 'Mas.Vnr.Area', 'Exter.Qual',
       'Exter.Cond', 'Foundation', 'Bsmt.Qual', 'Bsmt.Cond', 'Bsmt.Exposure',
       'BsmtFin.Type.1', 'BsmtFin.SF.1', 'BsmtFin.Type.2', 'BsmtFin.SF.2',
       'Bsmt.Unf.SF', 'Total.Bsmt.SF', 'Heating', 'Heating.QC', 'Central.Air',
       'Electrical', 'X1st.Flr.SF', 'X2nd.Flr.SF', 'Low.Qual.Fin.SF',
       'Gr.Liv.Area', 'Bsmt.Full.Bath', 'Bsmt.Half.Bath', 'Full.Bath',
       'Half.Bath', 'Bedroom.AbvGr', 'Kitchen.AbvGr', 'Kitchen.Qual',
       'TotRms.AbvGrd', 'Functional', 'Fireplaces', 'Fireplace.Qu',
       'Garage.Type', 'Garage.Yr.Blt', 'Garage.Finis

In [4]:
#that was a lot of columns...
#using this block to look at various columns to see what they include. Looking at columns whose dtypes are "object" 
#now look for any columns with nans so we can deal with them
nans = data.columns[data.isna().any()].tolist()
print(len(nans))
print(nans) #having nans in our dataframe really messes with the fit, lets find all columns with nans in them
qc_data = data.drop(nans, axis=1) #for now, lets just get rid of any columns with nans. 'qc' stands for quickly cleaned.
#we may want to come back later and maybe coerce these nans instead, so instead of modifying the original dataframe
#lets just make a new one.
qc_data.shape

26
['Lot.Frontage', 'Alley', 'Mas.Vnr.Type', 'Mas.Vnr.Area', 'Bsmt.Qual', 'Bsmt.Cond', 'Bsmt.Exposure', 'BsmtFin.Type.1', 'BsmtFin.SF.1', 'BsmtFin.Type.2', 'BsmtFin.SF.2', 'Bsmt.Unf.SF', 'Total.Bsmt.SF', 'Bsmt.Full.Bath', 'Bsmt.Half.Bath', 'Fireplace.Qu', 'Garage.Type', 'Garage.Yr.Blt', 'Garage.Finish', 'Garage.Cars', 'Garage.Area', 'Garage.Qual', 'Garage.Cond', 'Pool.QC', 'Fence', 'Misc.Feature']


(2344, 55)

In [5]:
#find what kinds of data have nans in them
nans_df = data[nans]
nans_df.dtypes

Lot.Frontage      float64
Alley              object
Mas.Vnr.Type       object
Mas.Vnr.Area      float64
Bsmt.Qual          object
Bsmt.Cond          object
Bsmt.Exposure      object
BsmtFin.Type.1     object
BsmtFin.SF.1      float64
BsmtFin.Type.2     object
BsmtFin.SF.2      float64
Bsmt.Unf.SF       float64
Total.Bsmt.SF     float64
Bsmt.Full.Bath    float64
Bsmt.Half.Bath    float64
Fireplace.Qu       object
Garage.Type        object
Garage.Yr.Blt     float64
Garage.Finish      object
Garage.Cars       float64
Garage.Area       float64
Garage.Qual        object
Garage.Cond        object
Pool.QC            object
Fence              object
Misc.Feature       object
dtype: object

I reordered the dataframe, putting saleprice as the first column, because we are going to need to one-hot encode several variables. Usually those extra columns end up being put the end of the dataframe, so that's why saleprice should be column 0, so we can easily access it so we can make our predictions.

I determined all of the columns with null values, so we need to fill this data in, rather than lose it. I do take the approach of deleting all columns with null values though, just to test it. Some columns with null values are filled with dtypes of floats, and some are filled with 'objects' or strings (see above output). Looking at that list of columns to dtypes, found a few columns where it seemed to make more sense to impute the constant zero instead of the column mean. That list is directly below. The rest of the columns that had floats I decided to use the mean, and for any column filled with strings, I used the mode. The mode is the only metric I can think  of that would work with a categorical variable, and I chose the mean because it seems like the more standard option compared to the median.

In [6]:
#Looking at the above columns, I think we should replace some of the missing values with 0 instead of the mean or median
#For example, Garage.Area. If Garage.Area has a NaN value, the house probably doesnt have a garage, so the value should be 0.
#list of columns where we want the NaNs replaced with zeroes
cols_0 = ['Lot.Frontage', 'Mas.Vnr.Area', 'BsmtFin.SF.1', 'BsmtFin.SF.2', 'Bsmt.Unf.SF', 'Total.Bsmt.SF', 'Garage.Area']

In [7]:
#create imputers
imp_0 = impute.SimpleImputer(missing_values = np.nan, strategy='constant', fill_value = 0, copy = False)
imp_mean = impute.SimpleImputer(missing_values = np.nan, strategy='mean', copy = False)
imp_mode = impute.SimpleImputer(missing_values = np.nan, strategy='most_frequent', copy = False)

In [8]:
#Lets attempt to look and find columns that have no bearing on price, and delete them.
#It is very hard to tell for many columns, but the first one seems to be a simple ID number. This would obviously have no 
#bearing on price so let's delete it.
del qc_data['PID']
del data['PID']
#next lets use a loop to one-hot encode all of the categorical variables
def cat_features(dataframe): #still need to move this into another file
    td = pd.DataFrame({'a':[1,2,3],'b':[4.0,5.0,6.0]})
    return filter(lambda x: not(dataframe[x].dtype in [td['a'].dtype, td['b'].dtype]), list(dataframe))

cat_feats = list(cat_features(data))
cat_feats2 = list(cat_features(qc_data))
def num_features(dataframe):
    td = pd.DataFrame({'a':[1,2,3],'b':[4.0,5.0,6.0]})
    return filter(lambda x: (dataframe[x].dtype in [td['a'].dtype, td['b'].dtype]), list(dataframe))

num_feats = list(num_features(data))

In [9]:
data[cols_0] = imp_0.fit_transform(data[cols_0])
data[cat_feats] = imp_mode.fit_transform(data[cat_feats])
data[num_feats] = imp_mean.fit_transform(data[num_feats])



Next, I one-hot encoded all of the categorical variables. One-hot encoding is generally a best practice for dealing with categorical variables where you don't want to give an 'increased value' to certain categories. I don't really have any prior knowledge of real estate, so none of the variables looked like they would benefit from label encoding to me.

In [10]:
#one-hot encode EVERYTHING!
enc_qc_data = pd.get_dummies(qc_data, columns = cat_feats2) #I don't want to get rid of the original dataframe, so I'll name this enc_data (short for encoded data)
enc_qc_data.head()

,SalePrice,MS.SubClass,Lot.Area,Overall.Qual,Overall.Cond,Year.Built,Year.Remod.Add,X1st.Flr.SF,X2nd.Flr.SF,Low.Qual.Fin.SF,Gr.Liv.Area,Full.Bath,Half.Bath,Bedroom.AbvGr,Kitchen.AbvGr,TotRms.AbvGrd,Fireplaces,Wood.Deck.SF,Open.Porch.SF,Enclosed.Porch,X3Ssn.Porch,Screen.Porch,Pool.Area,Misc.Val,Mo.Sold,Yr.Sold,MS.Zoning_A (agr),MS.Zoning_C (all),MS.Zoning_FV,MS.Zoning_I (all),MS.Zoning_RH,MS.Zoning_RL,MS.Zoning_RM,Street_Grvl,Street_Pave,Lot.Shape_IR1,Lot.Shape_IR2,Lot.Shape_IR3,Lot.Shape_Reg,Land.Contour_Bnk,Land.Contour_HLS,Land.Contour_Low,Land.Contour_Lvl,Utilities_AllPub,Utilities_NoSeWa,Utilities_NoSewr,Lot.Config_Corner,Lot.Config_CulDSac,Lot.Config_FR2,Lot.Config_FR3,Lot.Config_Inside,Land.Slope_Gtl,Land.Slope_Mod,Land.Slope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition.1_Artery,Condition.1_Feedr,Condition.1_Norm,Condition.1_PosA,Condition.1_PosN,Condition.1_RRAe,Condition.1_RRAn,Condition.1_RRNe,Condition.1_RRNn,Condition.2_Artery,Condition.2_Feedr,Condition.2_Norm,Condition.2_PosA,Condition.2_PosN,Condition.2_RRAe,Condition.2_RRNn,Bldg.Type_1Fam,Bldg.Type_2fmCon,Bldg.Type_Duplex,Bldg.Type_Twnhs,Bldg.Type_TwnhsE,House.Style_1.5Fin,House.Style_1.5Unf,House.Style_1Story,House.Style_2.5Fin,House.Style_2.5Unf,House.Style_2Story,House.Style_SFoyer,House.Style_SLvl,Roof.Style_Flat,Roof.Style_Gable,Roof.Style_Gambrel,Roof.Style_Hip,Roof.Style_Mansard,Roof.Style_Shed,Roof.Matl_ClyTile,Roof.Matl_CompShg,Roof.Matl_Membran,Roof.Matl_Roll,Roof.Matl_Tar&Grv,Roof.Matl_WdShake,Roof.Matl_WdShngl,Exterior.1st_AsbShng,Exterior.1st_AsphShn,Exterior.1st_BrkComm,Exterior.1st_BrkFace,Exterior.1st_CBlock,Exterior.1st_CemntBd,Exterior.1st_HdBoard,Exterior.1st_ImStucc,Exterior.1st_MetalSd,Exterior.1st_Plywood,Exterior.1st_PreCast,Exterior.1st_Stone,Exterior.1st_Stucco,Exterior.1st_VinylSd,Exterior.1st_Wd Sdng,Exterior.1st_WdShing,Exterior.2nd_AsbShng,Exterior.2nd_AsphShn,Exterior.2nd_Brk Cmn,Exterior.2nd_BrkFace,Exterior.2nd_CBlock,Exterior.2nd_CmentBd,Exterior.2nd_HdBoard,Exterior.2nd_ImStucc,Exterior.2nd_MetalSd,Exterior.2nd_Other,Exterior.2nd_Plywood,Exterior.2nd_PreCast,Exterior.2nd_Stone,Exterior.2nd_Stucco,Exterior.2nd_VinylSd,Exterior.2nd_Wd Sdng,Exterior.2nd_Wd Shng,Exter.Qual_Ex,Exter.Qual_Fa,Exter.Qual_Gd,Exter.Qual_TA,Exter.Cond_Ex,Exter.Cond_Fa,Exter.Cond_Gd,Exter.Cond_Po,Exter.Cond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,Heating.QC_Ex,Heating.QC_Fa,Heating.QC_Gd,Heating.QC_Po,Heating.QC_TA,Central.Air_N,Central.Air_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,Kitchen.Qual_Ex,Kitchen.Qual_Fa,Kitchen.Qual_Gd,Kitchen.Qual_TA,Functional_Maj1,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sal,Functional_Sev,Functional_Typ,Paved.Drive_N,Paved.Drive_P,Paved.Drive_Y,Sale.Type_COD,Sale.Type_CWD,Sale.Type_Con,Sale.Type_ConLD,Sale.Type_ConLI,Sale.Type_ConLw,Sale.Type_New,Sale.Type_Oth,Sale.Type_WD,Sale.Condition_Abnorml,Sale.Condition_AdjLand,Sale.Condition_Alloca,Sale.Condition_Family,Sale.Condition_Normal,Sale.Condition_Partial
0,105000,20,11622,5,6,1961,1961,896,0,0,896,1,0,2,1,5,0,140,0,0,0,120,0,0,6,2010,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0

In [11]:
#one-hot encode EVERYTHING!
data = pd.get_dummies(data, columns = cat_feats) 
data.head()

,SalePrice,MS.SubClass,Lot.Frontage,Lot.Area,Overall.Qual,Overall.Cond,Year.Built,Year.Remod.Add,Mas.Vnr.Area,BsmtFin.SF.1,BsmtFin.SF.2,Bsmt.Unf.SF,Total.Bsmt.SF,X1st.Flr.SF,X2nd.Flr.SF,Low.Qual.Fin.SF,Gr.Liv.Area,Bsmt.Full.Bath,Bsmt.Half.Bath,Full.Bath,Half.Bath,Bedroom.AbvGr,Kitchen.AbvGr,TotRms.AbvGrd,Fireplaces,Garage.Yr.Blt,Garage.Cars,Garage.Area,Wood.Deck.SF,Open.Porch.SF,Enclosed.Porch,X3Ssn.Porch,Screen.Porch,Pool.Area,Misc.Val,Mo.Sold,Yr.Sold,MS.Zoning_A (agr),MS.Zoning_C (all),MS.Zoning_FV,MS.Zoning_I (all),MS.Zoning_RH,MS.Zoning_RL,MS.Zoning_RM,Street_Grvl,Street_Pave,Alley_Grvl,Alley_Pave,Lot.Shape_IR1,Lot.Shape_IR2,Lot.Shape_IR3,Lot.Shape_Reg,Land.Contour_Bnk,Land.Contour_HLS,Land.Contour_Low,Land.Contour_Lvl,Utilities_AllPub,Utilities_NoSeWa,Utilities_NoSewr,Lot.Config_Corner,Lot.Config_CulDSac,Lot.Config_FR2,Lot.Config_FR3,Lot.Config_Inside,Land.Slope_Gtl,Land.Slope_Mod,Land.Slope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition.1_Artery,Condition.1_Feedr,Condition.1_Norm,Condition.1_PosA,Condition.1_PosN,Condition.1_RRAe,Condition.1_RRAn,Condition.1_RRNe,Condition.1_RRNn,Condition.2_Artery,Condition.2_Feedr,Condition.2_Norm,Condition.2_PosA,Condition.2_PosN,Condition.2_RRAe,Condition.2_RRNn,Bldg.Type_1Fam,Bldg.Type_2fmCon,Bldg.Type_Duplex,Bldg.Type_Twnhs,Bldg.Type_TwnhsE,House.Style_1.5Fin,House.Style_1.5Unf,House.Style_1Story,House.Style_2.5Fin,House.Style_2.5Unf,House.Style_2Story,House.Style_SFoyer,House.Style_SLvl,Roof.Style_Flat,Roof.Style_Gable,Roof.Style_Gambrel,Roof.Style_Hip,Roof.Style_Mansard,Roof.Style_Shed,Roof.Matl_ClyTile,Roof.Matl_CompShg,Roof.Matl_Membran,Roof.Matl_Roll,Roof.Matl_Tar&Grv,Roof.Matl_WdShake,Roof.Matl_WdShngl,Exterior.1st_AsbShng,Exterior.1st_AsphShn,Exterior.1st_BrkComm,Exterior.1st_BrkFace,Exterior.1st_CBlock,Exterior.1st_CemntBd,Exterior.1st_HdBoard,Exterior.1st_ImStucc,Exterior.1st_MetalSd,Exterior.1st_Plywood,Exterior.1st_PreCast,Exterior.1st_Stone,Exterior.1st_Stucco,Exterior.1st_VinylSd,Exterior.1st_Wd Sdng,Exterior.1st_WdShing,Exterior.2nd_AsbShng,Exterior.2nd_AsphShn,Exterior.2nd_Brk Cmn,Exterior.2nd_BrkFace,Exterior.2nd_CBlock,Exterior.2nd_CmentBd,Exterior.2nd_HdBoard,Exterior.2nd_ImStucc,Exterior.2nd_MetalSd,Exterior.2nd_Other,Exterior.2nd_Plywood,Exterior.2nd_PreCast,Exterior.2nd_Stone,Exterior.2nd_Stucco,Exterior.2nd_VinylSd,Exterior.2nd_Wd Sdng,Exterior.2nd_Wd Shng,Mas.Vnr.Type_BrkCmn,Mas.Vnr.Type_BrkFace,Mas.Vnr.Type_None,Mas.Vnr.Type_Stone,Exter.Qual_Ex,Exter.Qual_Fa,Exter.Qual_Gd,Exter.Qual_TA,Exter.Cond_Ex,Exter.Cond_Fa,Exter.Cond_Gd,Exter.Cond_Po,Exter.Cond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Bsmt.Qual_Ex,Bsmt.Qual_Fa,Bsmt.Qual_Gd,Bsmt.Qual_Po,Bsmt.Qual_TA,Bsmt.Cond_Ex,Bsmt.Cond_Fa,Bsmt.Cond_Gd,Bsmt.Cond_Po,Bsmt.Cond_TA,Bsmt.Exposure_Av,Bsmt.Exposure_Gd,Bsmt.Exposure_Mn,Bsmt.Exposure_No,BsmtFin.Type.1_ALQ,BsmtFin.Type.1_BLQ,BsmtFin.Type.1_GLQ,BsmtFin.Type.1_LwQ,BsmtFin.Type.1_Rec,BsmtFin.Type.1_Unf,BsmtFin.Type.2_ALQ,BsmtFin.Type.2_BLQ,BsmtFin.Type.2_GLQ,BsmtFin.Type.2_LwQ,BsmtFin.Type.2_Rec,BsmtFin.Type.2_Unf,Heating_Floor,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,Heating.QC_Ex,Heating.QC_Fa,Heating.QC_Gd,Heating.QC_Po,Heating.QC_TA,Central.Air_N,Central.Air_Y,Electrical_FuseA,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,Kitchen.Qual_Ex,Kitchen.Qual_Fa,Kitchen.Qual_Gd,Kitchen.Qual_TA,Functional_Maj1,Functional_Maj2,Fu

## Model-Building
### Create baseline model! (No Feature Selection or Regularization)

In [12]:
#now that all our categorical variables are done, its time to make a baseline model.
data_y = enc_qc_data['SalePrice']
data_x = enc_qc_data[list(enc_qc_data)[1:]]
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y,test_size = 0.2, random_state = 4) #create test and training data

#build model, fit on test data, and create predictions
base_model = linear_model.LinearRegression()#make model object
base_model.fit(x_train, y_train)#fit the model (using all features, except ones that were deleted)
preds = base_model.predict(x_test)#build prediction on the test data
print('R^2 (Baseline Model): ' + str(r2_score(y_test, preds)))

R^2 (Baseline Model): 0.8586743971596199


Not Bad, especially for a baseline! With our baseline, even with missing data able to get such a high r2 value, I doubt that the polynomial regression is the way to go. I'm going to stick with linear regression for this project.

In [13]:
#do it up
data_y_2 = data['SalePrice']
data_x_2 = data[list(data)[1:]]
x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(data_x_2, data_y_2,test_size = 0.2, random_state = 4) #create test and training data

#build model, fit on test data, and create predictions
model = linear_model.LinearRegression()#make model object
model.fit(x_train_2, y_train_2)#fit the model 
preds_2 = model.predict(x_test_2)#build prediction on the test data
print('R^2 (Baseline with Imputing): ' + str(r2_score(y_test_2, preds_2)))

R^2 (Baseline with Imputing): 0.9014567426545751


Hey, nice! Our accuracy metric went up! Usually using an imputer will increase accuracy by a bit because we are inputing
means, which fits nicely on a regression line, so this may be a false positive. Just something to keep in mind. Now that we have done our transforms and dealt with our missing data, let's try out some feature selection with cross-validation.

### Adding in feature selection
Now that we have all of our data set up we need to select which features to use to optimize performance.

In [14]:
selector_f = RFECV(estimator=linear_model.LinearRegression(), cv=10, scoring=make_scorer(r2_score))
selector_f.fit(x_train_2, y_train_2)

xt_train, xt_test = selector_f.transform(x_train_2), selector_f.transform(x_test_2)
fs_model = linear_model.LinearRegression()
fs_model.fit(xt_train, y_train_2)
fs_preds = fs_model.predict(xt_test)
print('R^2 (With Imputing and Feature Selection): ' + str(r2_score(y_test_2, fs_preds)))

R^2 (With Imputing and Feature Selection): 0.6221185941158986


That... wasn't what I was expecting. I don't think I made a mistake anyhwere... best I can guess is that all the imputions artifically raised the r2 by a LOT. That still doesn't explain why it performs worse than the original baseline though.

## Adding in regularization 
What will happen?

In [15]:
alphas = [0.0, 0.01, 0.1, 0.25, 0.5, 1.0, 2.5 , 5, 7, 11]
lasso_model = linear_model.Lasso(alpha = 2.5, normalize = True, fit_intercept = True)
lasso_model.fit(x_train, y_train)
preds = lasso_model.predict(x_test)
print('R^2 (Lasso Model With Alpha= 2.5 ):  ' + str(r2_score(y_test, preds)))

R^2 (Lasso Model With Alpha= 2.5 ):  0.8641775844310426


That is a lot of warnings. And the best we managed to do was get .01 higher than our original model. Let's try this out on our imputed set and our feature-selected set now.

In [16]:
lasso_model2 = linear_model.Lasso(alpha = 2.5, normalize = True, fit_intercept = True)
lasso_model2.fit(x_train_2, y_train_2)
preds = lasso_model2.predict(x_test_2)
print('R^2 (Lasso Model With Alpha= 2.5 ' '):  ' + str(r2_score(y_test_2, preds)))

R^2 (Lasso Model With Alpha= 2.5 ):  0.9058737610959191


In [17]:
lasso_model3 = linear_model.Lasso(alpha = 2.5, normalize = True, fit_intercept = True)
lasso_model3.fit(xt_train, y_train_2)
preds = lasso_model3.predict(xt_test)
print('R^2 (Lasso Model With Alpha= 2.5 ):'   + str(r2_score(y_test_2, preds)))

R^2 (Lasso Model With Alpha= 2.5 ):0.6215818282464154


C:\Users\xxzac\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


Using loops, (not shown) it looks like an alpha of 2.5 has the best performance for the feature selected set, but performance barely increased. While the selected set didn't work out for dataset A, maybe it will work best on B?

# Testing models on set B

In [18]:
datab = pd.read_csv('AmesHousingSetB.csv')
datab = datab[['SalePrice', 'PID', 'MS.SubClass', 'MS.Zoning', 'Lot.Frontage', 'Lot.Area', 'Street',
       'Alley', 'Lot.Shape', 'Land.Contour', 'Utilities', 'Lot.Config',
       'Land.Slope', 'Neighborhood', 'Condition.1', 'Condition.2', 'Bldg.Type',
       'House.Style', 'Overall.Qual', 'Overall.Cond', 'Year.Built',
       'Year.Remod.Add', 'Roof.Style', 'Roof.Matl', 'Exterior.1st',
       'Exterior.2nd', 'Mas.Vnr.Type', 'Mas.Vnr.Area', 'Exter.Qual',
       'Exter.Cond', 'Foundation', 'Bsmt.Qual', 'Bsmt.Cond', 'Bsmt.Exposure',
       'BsmtFin.Type.1', 'BsmtFin.SF.1', 'BsmtFin.Type.2', 'BsmtFin.SF.2',
       'Bsmt.Unf.SF', 'Total.Bsmt.SF', 'Heating', 'Heating.QC', 'Central.Air',
       'Electrical', 'X1st.Flr.SF', 'X2nd.Flr.SF', 'Low.Qual.Fin.SF',
       'Gr.Liv.Area', 'Bsmt.Full.Bath', 'Bsmt.Half.Bath', 'Full.Bath',
       'Half.Bath', 'Bedroom.AbvGr', 'Kitchen.AbvGr', 'Kitchen.Qual',
       'TotRms.AbvGrd', 'Functional', 'Fireplaces', 'Fireplace.Qu',
       'Garage.Type', 'Garage.Yr.Blt', 'Garage.Finish', 'Garage.Cars',
       'Garage.Area', 'Garage.Qual', 'Garage.Cond', 'Paved.Drive',
       'Wood.Deck.SF', 'Open.Porch.SF', 'Enclosed.Porch', 'X3Ssn.Porch',
       'Screen.Porch', 'Pool.Area', 'Pool.QC', 'Fence', 'Misc.Feature',
       'Misc.Val', 'Mo.Sold', 'Yr.Sold', 'Sale.Type', 'Sale.Condition']]
del datab['PID']
print(datab.shape)

(586, 80)


In [19]:
nansb = datab.columns[datab.isna().any()].tolist()
print(len(nansb))
print(nansb) #having nans in our dataframe really messes with the fit, lets find all columns with nans in them
qc_datab = datab.drop(nansb, axis=1)
print(qc_datab.shape)

19
['Lot.Frontage', 'Alley', 'Mas.Vnr.Type', 'Mas.Vnr.Area', 'Bsmt.Qual', 'Bsmt.Cond', 'Bsmt.Exposure', 'BsmtFin.Type.1', 'BsmtFin.Type.2', 'Electrical', 'Fireplace.Qu', 'Garage.Type', 'Garage.Yr.Blt', 'Garage.Finish', 'Garage.Qual', 'Garage.Cond', 'Pool.QC', 'Fence', 'Misc.Feature']
(586, 61)


In [20]:
#del qc_datab['PID']
#del datab['PID']
#next lets use a loop to one-hot encode all of the categorical variables


cat_featsb = list(cat_features(datab))
cat_featsb2 = list(cat_features(qc_datab))


num_featsb = list(num_features(datab))

In [21]:
datab[cols_0] = imp_0.fit_transform(datab[cols_0])
datab[cat_featsb] = imp_mode.fit_transform(datab[cat_featsb])
datab[num_featsb] = imp_mean.fit_transform(datab[num_featsb])


In [22]:
enc_qc_datab = pd.get_dummies(qc_datab, columns = cat_featsb2) #I don't want to get rid of the original dataframe, so I'll name this enc_data (short for encoded data)
enc_qc_datab.head()

,SalePrice,MS.SubClass,Lot.Area,Overall.Qual,Overall.Cond,Year.Built,Year.Remod.Add,BsmtFin.SF.1,BsmtFin.SF.2,Bsmt.Unf.SF,Total.Bsmt.SF,X1st.Flr.SF,X2nd.Flr.SF,Low.Qual.Fin.SF,Gr.Liv.Area,Bsmt.Full.Bath,Bsmt.Half.Bath,Full.Bath,Half.Bath,Bedroom.AbvGr,Kitchen.AbvGr,TotRms.AbvGrd,Fireplaces,Garage.Cars,Garage.Area,Wood.Deck.SF,Open.Porch.SF,Enclosed.Porch,X3Ssn.Porch,Screen.Porch,Pool.Area,Misc.Val,Mo.Sold,Yr.Sold,MS.Zoning_C (all),MS.Zoning_FV,MS.Zoning_I (all),MS.Zoning_RH,MS.Zoning_RL,MS.Zoning_RM,Street_Grvl,Street_Pave,Lot.Shape_IR1,Lot.Shape_IR2,Lot.Shape_IR3,Lot.Shape_Reg,Land.Contour_Bnk,Land.Contour_HLS,Land.Contour_Low,Land.Contour_Lvl,Utilities_AllPub,Lot.Config_Corner,Lot.Config_CulDSac,Lot.Config_FR2,Lot.Config_FR3,Lot.Config_Inside,Land.Slope_Gtl,Land.Slope_Mod,Land.Slope_Sev,Neighborhood_Blmngtn,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition.1_Artery,Condition.1_Feedr,Condition.1_Norm,Condition.1_PosA,Condition.1_PosN,Condition.1_RRAe,Condition.1_RRAn,Condition.1_RRNn,Condition.2_Artery,Condition.2_Feedr,Condition.2_Norm,Condition.2_RRAn,Bldg.Type_1Fam,Bldg.Type_2fmCon,Bldg.Type_Duplex,Bldg.Type_Twnhs,Bldg.Type_TwnhsE,House.Style_1.5Fin,House.Style_1.5Unf,House.Style_1Story,House.Style_2.5Fin,House.Style_2.5Unf,House.Style_2Story,House.Style_SFoyer,House.Style_SLvl,Roof.Style_Flat,Roof.Style_Gable,Roof.Style_Gambrel,Roof.Style_Hip,Roof.Style_Mansard,Roof.Style_Shed,Roof.Matl_CompShg,Roof.Matl_Metal,Roof.Matl_Tar&Grv,Roof.Matl_WdShake,Roof.Matl_WdShngl,Exterior.1st_AsbShng,Exterior.1st_BrkFace,Exterior.1st_CemntBd,Exterior.1st_HdBoard,Exterior.1st_MetalSd,Exterior.1st_Plywood,Exterior.1st_Stucco,Exterior.1st_VinylSd,Exterior.1st_Wd Sdng,Exterior.1st_WdShing,Exterior.2nd_AsbShng,Exterior.2nd_AsphShn,Exterior.2nd_Brk Cmn,Exterior.2nd_BrkFace,Exterior.2nd_CmentBd,Exterior.2nd_HdBoard,Exterior.2nd_ImStucc,Exterior.2nd_MetalSd,Exterior.2nd_Plywood,Exterior.2nd_Stone,Exterior.2nd_Stucco,Exterior.2nd_VinylSd,Exterior.2nd_Wd Sdng,Exterior.2nd_Wd Shng,Exter.Qual_Ex,Exter.Qual_Fa,Exter.Qual_Gd,Exter.Qual_TA,Exter.Cond_Ex,Exter.Cond_Fa,Exter.Cond_Gd,Exter.Cond_Po,Exter.Cond_TA,Foundation_BrkTil,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_GasA,Heating_GasW,Heating_Grav,Heating_Wall,Heating.QC_Ex,Heating.QC_Fa,Heating.QC_Gd,Heating.QC_Po,Heating.QC_TA,Central.Air_N,Central.Air_Y,Kitchen.Qual_Ex,Kitchen.Qual_Fa,Kitchen.Qual_Gd,Kitchen.Qual_Po,Kitchen.Qual_TA,Functional_Maj1,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Typ,Paved.Drive_N,Paved.Drive_P,Paved.Drive_Y,Sale.Type_COD,Sale.Type_CWD,Sale.Type_ConLD,Sale.Type_ConLI,Sale.Type_ConLw,Sale.Type_New,Sale.Type_Oth,Sale.Type_VWD,Sale.Type_WD,Sale.Condition_Abnorml,Sale.Condition_AdjLand,Sale.Condition_Alloca,Sale.Condition_Family,Sale.Condition_Normal,Sale.Condition_Partial
0,252000,60,10790,7,5,1998,1998,0,0,1066,1066,1108,1277,0,2385,0,0,2,1,4,1,8,1,3,600,120,38,0,0,0,0,0,5,2007,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,256300,20,14230,8,5,2007,2007,0,0,1566,1566,1600,0,0,1600,0,0,2,0,3,1,7,1,3,890,0,56,0,0,0,0,0,7,2009,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0

In [23]:
#datab = pd.get_dummies(datab, columns = cat_featsb) 
datab = pd.get_dummies(datab, columns = cat_featsb)
datab.head()
datab.shape

(586, 270)

In [24]:
colsa = list(data.columns)
colsb = list(datab.columns)
diff = np.setdiff1d(colsa,colsb)
for i in diff:
    del data[i]
diff2 = np.setdiff1d(colsb, colsa)
print
for i in diff2:
    del datab[i]

In [25]:
enccolsa = list(enc_qc_data.columns)
enccolsb = list(enc_qc_datab.columns)
encdiff = np.setdiff1d(enccolsa,enccolsb)
for i in encdiff:
    del enc_qc_data[i]
encdiff2 = np.setdiff1d(enccolsb, enccolsa)
for i in encdiff2:
    del enc_qc_datab[i]

In [26]:
data_yb = enc_qc_datab['SalePrice']
data_xb = enc_qc_datab[list(enc_qc_datab)[1:]]
x_trainb, x_testb, y_trainb, y_testb = train_test_split(data_xb, data_yb,test_size = 0.2, random_state = 4)

In [34]:
#need to resplit and retrain everything because of data points unique to the b dataset. Making sure to use the A data though!
data_y = enc_qc_data['SalePrice']
data_x = enc_qc_data[list(enc_qc_data)[1:]]
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y,test_size = 0.2, random_state = 4)
base_model.fit(x_train, y_train)
data_y_2 = data['SalePrice']
data_x_2 = data[list(data)[1:]]
x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(data_x_2, data_y_2,test_size = 0.2, random_state = 4)
model.fit(x_train_2, y_train_2)
lasso_model.fit(x_train, y_train)
lasso_model2.fit(x_train_2, y_train_2)
selector_f.fit(x_train_2, y_train_2)
xt_train, xt_test = selector_f.transform(x_train_2), selector_f.transform(x_test_2)
fs_model.fit(xt_train, y_train_2)
lasso_model3.fit(xt_train, y_train_2)

Lasso(alpha=2.5, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=True, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [39]:
xt_train.shape

(1875, 263)

In [28]:

preds = base_model.predict(x_testb)#build prediction on the test data
print('R^2 (Baseline Model): ' + str(r2_score(y_testb, preds)))



R^2 (Baseline Model): 0.9090262096052781


FINALLY I HAVE A USABLE (?) OUTPUT!

In [29]:
datab_y_2 = datab['SalePrice']
datab_x_2 = datab[list(datab)[1:]]
x_train_2b, x_test_2b, y_train_2b, y_test_2b = train_test_split(datab_x_2, datab_y_2,test_size = 0.2, random_state = 4) #create test and training data

#build model, fit on test data, and create predictions
preds_2b = model.predict(x_test_2b)#build prediction on the test data
print('R^2 (Baseline with Imputing): ' + str(r2_score(y_test_2b, preds_2b)))

R^2 (Baseline with Imputing): -26.792988518535903


Wow. That is... shockingly bad. Its unintelligable

In [30]:
xt_trainb, xt_testb = selector_f.transform(x_train_2b), selector_f.transform(x_test_2b)
fs_predsb = fs_model.predict(xt_testb)
print('R^2 (With Imputing and Feature Selection): ' + str(r2_score(y_test_2b, fs_predsb)))

R^2 (With Imputing and Feature Selection): -26.792988518535903


...weird. Another unintelligable result. Maybe I was supposed to refit the selector to the B dataset? If I did then I doubt that it would fit the model though. Also it looks like the same unintelligable result as last time...?

In [31]:
preds = lasso_model.predict(x_testb)
print('R^2 (Lasso Model With Alpha= 2.5 ):'   + str(r2_score(y_testb, preds)))

R^2 (Lasso Model With Alpha= 2.5 ):0.9187145015345759


Hey we got a real value this time. 

In [32]:
preds = lasso_model2.predict(x_test_2b)
print('R^2 (Lasso Model With Alpha= 2.5 ): ' + str(r2_score(y_test_2b, preds)))

R^2 (Lasso Model With Alpha= 2.5 ): -24.601166214998045


That was shortlived...

In [42]:
preds = lasso_model3.predict(xt_testb)
print('R^2 (Lasso Model With Alpha= 2.5 ): ' + str(r2_score(y_test_2b, preds)))

R^2 (Lasso Model With Alpha= 2.5 ): -24.601166214998045


There it is again...